## Helpful code to analize crypto

### Analize the result of paired trading

In this scenario the pair will maintain a balance between value and quote value according with the weight. The rebalance will occur in each candle, so attention on the interval that was chosen in get_data function.

#### Example:

For the pair BTCUSDT (value = BTC, quote_value = USDT);

Total value for the analysis: $ 50000.00;

Weight: 2;


For each candle, the total value will be divided by the weight, and the rebalance will be perfomed. Is limit_quote_value is setted to True, the rebalance only occurs when the new quote value is greater then the past quote value.

In [ ]:
import pandas as pd
import numpy as np
from binance_utils import init
from trade_utils import get_data

total_value = 50000.00
weight = 2
# limit_quote_value
# True: only rebalance is new quote_value is greater than previous.
# False: always rebalance 
limit_quote_loss = False

client_live = init()
data = get_data(client_live, 'BTCBRL', '1M', save=False)
data = data['2021-09-09':]
data.reset_index(inplace=True)
data['result'] = 0.0
data['value'] = 0.0
data['quoteValue'] = 0.0
data['benchmark'] = 0.0

initial_quote_value = total_value / weight
initial_value = initial_quote_value / data.iloc[0]['ClosePrice']
benchmark_value = total_value / data.iloc[0]['ClosePrice']

for i in range(len(data)):
    if i == 0:
        data.loc[i, ['result']] = initial_quote_value * weight
    else:
        data.loc[i, ['result']] = (data.iloc[i - 1]['value'] * data.iloc[i]['ClosePrice']) + data.iloc[i - 1]['quoteValue']
    
    quote_value = data.iloc[i]['result'] / weight
    if (quote_value < data.iloc[i - 1]['quoteValue'] and limit_quote_loss):
        quote_value = data.iloc[i - 1]['quoteValue']
    
    data.loc[i, ['quoteValue']] = quote_value
    data.loc[i, ['value']] = (data.iloc[i]['result'] - data.iloc[i]['quoteValue']) / data.iloc[i]['ClosePrice']
    data.loc[i, ['benchmark']] = benchmark_value * data.iloc[i]['ClosePrice']

data[['OpenTime', 'ClosePrice', 'value', 'quoteValue', 'result', 'benchmark']]

In [ ]:
data[data['quoteValue'].max()==data['quoteValue']]

In [ ]:
data[data['value'].max()==data['value']]

### Get all filled trades for a specific pair.

In [ ]:
from binance_utils import init, get_trades
import pandas as pd

client_live = init()

data = get_trades(client_live, 'FILBTC')
data

### Create on data frame with all coins

In [ ]:
import glob
import pandas as pd
import re

# Find all files with '*all*.parquet' in their name
files = glob.glob('/media/evandro/Work/Workspace/data/trading/*all*.parquet')

# Read each file and store its data in a dataframe with filename as the first column
dfs = []
for file in files:
    pair = re.search(r'trading/(.*?)-1m', file).group(1)
    df = pd.read_parquet(file)
    df.reset_index(inplace=True)
    df.insert(0, 'Pair', pair)
    dfs.append(df)

# Concatenate all dataframes into one
all_data = pd.concat(dfs, ignore_index=True)
all_data